## DBPIA 논문 크롤링
- 기본 페이지 외에 => 상세 페이지로 접근하는 로직에 대해 알 수 있습니다.

In [1]:
# libary import
from selenium import webdriver
from selenium.webdriver.common.by import By

import time # 시간 딜레이시 사용
import pandas as pd # 엑셀 저장시 사용

In [2]:
browser = webdriver.Chrome()

In [3]:
# 사이트 이동
url = 'https://www.dbpia.co.kr/search/topSearch?searchOption=all&query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5'
browser.get(url)

In [15]:
# 논문 링크만 쭉 가져오는 코드
# link = browser.find_element(By.CLASS_NAME, 'thesis__link').get_attribute('href')
datas = browser.find_elements(By.CLASS_NAME, 'thesis__link')

detail_url_list = []
for i in datas:
    detail_url = i.get_attribute('href')
    detail_url_list.append(detail_url)

print(detail_url_list)

['https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11230851', 'https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11230843', 'https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11230845', 'https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11230844', 'https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11288340', 'https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11231160', 'https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11231378', 'https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11288024', 'https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11231192', 'https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11230847', 'https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11337935', 'https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11230365', 'https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11342633', 'https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11231818', 'https://www.dbpia.co.kr/journal/

In [17]:
for link in detail_url_list:
    browser.get(link)
    time.sleep(3)

In [18]:
# 제목, 초록*키워드 가져와보시고
# => 초록이 없는 경우가 있어요. (이건 어떻게 할 수 있을까요?)
# - no such element가 뜨겠죠?
# -> try, except(오류 상황이 발생): 다음 페이지로 넘어가 => 코드가 지저분해지잖아..
# 다 하신분들은 => 3페이지까지도 해보시구요. (페이지 이동)

In [6]:
# 버튼을 클릭하는 부분
# //*[@id="pageList"]/a[2] - 2페이지
# //*[@id="pageList"]/a[3] - 3페이지
# //*[@id="pageList"]/a[10] - 10페이지

page = '//*[@id="pageList"]/a[{}]'

for i in range(1,4):
    final_page = page.format(i)
    print(final_page)

//*[@id="pageList"]/a[1]
//*[@id="pageList"]/a[2]
//*[@id="pageList"]/a[3]


In [16]:
# (1) DB pia 사이트로 간다.
# (2) 1페이지에 있는 논문 링크들을 다 가져온다.
# (3) 2페이지로 간다.
# (4) 2페이지에 있는 논문 링크들을 다 가져온다.
# (5) 3페이지로 간다.

page_xpath = '//*[@id="pageList"]/a[{}]'

link_list = []

for i in range(1,3):
    next_page_xpath = page_xpath.format(i)
    browser.find_element(By.XPATH, next_page_xpath).click()
    time.sleep(5)

    # 해당 페이지의 링크 값들을 다 가져와서 link_list에 저장을 한다.
    links = browser.find_elements(By.CLASS_NAME, 'thesis__link')

    for j in links:
        link = j.get_attribute('href')
        link_list.append(link)

In [23]:
import pandas as pd

num = 0

data_list = []
for link in link_list:
    num += 1
    if num == 10:
        break
    browser.get(link)
    time.sleep(2) # 무조건 5초를 기다려

    # 타이틀
    # title = browser.find_element(By.ID, 'thesisTitle').get_attribute('class') # -> thesisDetail__tit
    title = browser.find_element(By.ID, 'thesisTitle').text

    # 초록*키워드
    content = browser.find_element(By.CLASS_NAME, 'abstractTxt').text

    data = {
        "제목" : title,
        "초록-키워드" : content 
    }

    data_list.append(data)

df = pd.DataFrame(data_list)
df
df.to_csv("./dbpia_scholar.csv", encoding="utf-8-sig")

In [8]:
# https://www.google.com/search?q=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&tbm=nws&ei=SoVAZOndLMmx2roPyPOX8Ag&start=0 - 1페이지
# https://www.google.com/search?q=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&tbm=nws&ei=SoVAZOndLMmx2roPyPOX8Ag&start=10 - 2페이지
# https://www.google.com/search?q=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&tbm=nws&ei=U4VAZPbYKobf2roPmq-ayA4&start=20 - 3페이지
# https://www.google.com/search?q=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&tbm=nws&ei=eYVAZKjqNc3M2roP0NKSuAM&start=90 - 10페이지
# tbm => tab menu - nws(news) # GET data

url = "https://www.google.com/search?q=검색어&start={}"

for i in range(0,100, 10):
    final_url = url.format(i)
    print(final_url)

https://www.google.com/search?q=검색어&start=0
https://www.google.com/search?q=검색어&start=10
https://www.google.com/search?q=검색어&start=20
https://www.google.com/search?q=검색어&start=30
https://www.google.com/search?q=검색어&start=40
https://www.google.com/search?q=검색어&start=50
https://www.google.com/search?q=검색어&start=60
https://www.google.com/search?q=검색어&start=70
https://www.google.com/search?q=검색어&start=80
https://www.google.com/search?q=검색어&start=90


[0, 1, 2, 3]